# Regular Box Score Scrapper (Team Level)

In [92]:
#%run ../Componenets/00_ProjectImports.ipynb
#%run ../Componenets/01_UserAgentInformation.ipynb

In [100]:
def regularBoxScoreScraper(team,season):  
    '''
    INPUTS
    team - (string input) nba basketball team abbreviation, This should come from the TeamAbbreviation notebook
    season - (string input) This is a four character parameter yyyy
    ----------------------------------------------------------------
    OUTPUT
    df - this data frame contains advanced box score game logs
    '''
    #Creating our url string to go fetch website data
    url = 'https://www.basketball-reference.com/teams/'+team+'/'+season+'/gamelog/'
    #calling our user agent variables to create a request header
    headers = {"User-Agent":user_agent}
    #Initiating the web page and finding table contents
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.text, 'lxml')
    table = soup.find('div', {'class':'table_wrapper'})
    table_rows = table.find_all('tr')
    df = pd.DataFrame(pd.read_html(str(table))[0])
    df['Date'] =df[('Unnamed: 2_level_0',                'Date')]
    df['Location'] = df[('Unnamed: 3_level_0',                'Unnamed: 3_level_1')]
    df['Opposition'] =df[('Score',                'Opp')]
    df['Team_Points'] =df[('Score',                'Tm')]
    df['Opponent_Points'] =df[('Score',                'Opp')]
    df['FTA']=df[('Team',                'FTA')]
    df['FT_Perc']=df[('Team',                'FT%')]
    df['FG']=df[('Team',                'FG')]
    df['FGA_RAW']=df[('Team',                'FGA')]
    df['ThreeP']=df[('Team',                '3P')]
    df['ThreePA']=df[('Team',                '3PA')]
    df['ThreePA_Perc']=df[('Team',                '3P%')]
    df['Offensive_Rebounds']=df[('Team',                'ORB')]
    df['Total_Rebounds']=df[('Team',                'TRB')]
    df['Assists']=df[('Team',                'AST')]
    df['Steals']=df[('Team',                'STL')]
    df['Blocks']=df[('Team',                'BLK')]
    df['TurnOvers']=df[('Team',                'TOV')]
    df['Personal_Fouls']=df[('Team',                'PF')]
    #Elimiating the fake column header row
    df.columns = df.columns.droplevel(1)
    #The next list is making sure we are only keeping the data fields we care about
    df = df[[

            'Date',
            'Location',
            'Opposition',
            'Team_Points',
            'Opponent_Points',
            'FTA',
            'FT_Perc',
            'FG',
            'FGA_RAW',
            'ThreeP',
            'ThreePA',
            'ThreePA_Perc',
            'Offensive_Rebounds',
            'Total_Rebounds',
            'Assists',
            'Steals',
            'Blocks',
            'TurnOvers',
            'Personal_Fouls'
            ]]
    #Renamib the unnamed column headers
    df.rename(columns={ 'Opposition':'Opponent'
                        
                       }, inplace = True
              )
    df = df[df['Opponent'].notnull()]
    df = df[df.Opponent != 'Opp']
    df['Team'] = str(team)
    df['Points_Scored'] = df['Team_Points'].astype(int) + df['Opponent_Points'].astype(int)
    df['FGA'] = df['FGA_RAW'].astype(int) - df['ThreePA'].astype(int)
    
    df['FG_Perc'] = (df['FG'].astype(int) - df['ThreeP'].astype(int))/df['FGA'].astype(int)
    
    df["Location"].fillna("VS", inplace = True)
    df = df[[

            'Date',
            'Location',
            'Team',
            'Opponent',
            'Team_Points',
            'Opponent_Points',
            'Points_Scored',
            'FTA',
            'FT_Perc',
            'FG',
            'FGA',
            'FG_Perc',
            'ThreeP',
            'ThreePA',
            'ThreePA_Perc',
            'Offensive_Rebounds',
            'Total_Rebounds',
            'Assists',
            'Steals',
            'Blocks',
            'TurnOvers',
            'Personal_Fouls'
            ]]
    return df